In [1]:
import pandas as pd
import numpy as np
import random 

In [2]:
class visitor:
    def __init__(self, vid, building, numberOfFloors, roomsPerFloor,contagious):
        self.id = vid
        self.contagiousDegree = random.uniform(0.1,0.6) if contagious else 0
        # define number of visit
        numberOfVisits = random.randint(1, numberOfFloors * roomsPerFloor)
        visitedRooms = []
        self.itinerary = [self._decideRoom(building,numberOfFloors, roomsPerFloor, visitedRooms, None)\
                            for visit in range(1, numberOfVisits + 1)]

    def _decideRoom(self, building, numberOfFloors, roomsPerFloor, visitedRooms, currentFloor):
            while(True):
                floor = self._decideFloor(building, numberOfFloors, currentFloor)
                room = random.randint(1, roomsPerFloor)                
                if (floor, room) not in visitedRooms:
                    rid = -1
                    for rm in building:
                        if rm['room']==room and rm['floor']==floor:
                            rid = rm['rid']
                    visitedRooms.append((floor, room))
                    return {'rid':rid, 'floor':floor, 'room':room, 'visitorId':self.id, 'contagiousDegree':self.contagiousDegree}

    def _decideFloor(self, building, numberOfFloors: int, currentFloor: int):
            if currentFloor==None:  # no current floor, has to be first room in itinerary             
                # decide start floor
                return random.randint(1, numberOfFloors)
            else:
                # decide if stay in current floor or move to another floor
                stayInFloor = random.randint(0, 1)
                if stayInFloor == 1:
                    # stay in current floor
                    return currentFloor

                # else, move to another floor
                # decide if use elevator
                useElevator = random.randint(0, 1)
                if useElevator == 1:
                    # use elevator and move to any other floor in building (excpet current floor)
                    while (True):
                        nextFloor = random.randint(1, numberOfFloors)
                        if nextFloor != currentFloor:
                            return nextFloor

                # else, don't use elevator, use stairs instead
                goOneFloorUpstairs = random.randint(0, 1)
                if goOneFloorUpstairs == 0 and currentFloor != 1:
                    # don't go upstairs, go downstairs one floor
                    return currentFloor - 1
                    # else, go upstairs one floor
                return currentFloor + 1

In [43]:
roomsPerFloor = 4
numberOfFloors = 3
numberOfVisitor= 5

building = [{'rid': i, 'floor':f, 'room':r, 'susceptability':random.uniform(0,1)}\
            for i, (r,f) in enumerate(\
            [(room,floor)\
            for floor in range(1,numberOfFloors+1)\
            for room in range(1,roomsPerFloor+1)])]


contagiousVisitor = random.randint(1,numberOfVisitor)

itineraries = [visitor(vid,building, numberOfFloors, roomsPerFloor, contagiousVisitor==vid).itinerary\
          for vid in range(1,numberOfVisitor+1)]

infected = [{'rid':r['rid'], 'floor':visit['floor'], 'room':visit['room']}\
            for itinerary in itineraries
            for visit in itinerary
            for r in building
            if visit['contagiousDegree']>0 and
               r['floor']==visit['floor'] and
               r['room']==visit['room'] and
               r['susceptability']+visit['contagiousDegree'] > 0.9]


In [44]:
print('building')
pd.DataFrame(building)

building


,floor,rid,room,susceptability
0,1,0,1,0.935709
1,1,1,2,0.522779
2,1,2,3,0.849844
3,1,3,4,0.405771
4,2,4,1,0.304995
5,2,5,2,0.932260
6,2,6,3,0.016201
7,2,7,4,0.972097
8,3,8,1,0.394200
9,3,9,2,0.892570


In [45]:
print('itineraries')
print("\n".join(["rid: {} floor: {} room:{} vid: {} contagiousDegree: {}".format(
                 r["rid"],v["floor"], v["room"], v["visitorId"], v["contagiousDegree"]) 
                 for itinerary in itineraries
                 for v in itinerary
                 for r in building 
                 if r['room']==v['room'] and r['floor']==v['floor']
                 #if v["contagiousDegree"] >0
                ]))


itineraries
rid: 10 floor: 3 room:3 vid: 1 contagiousDegree: 0
rid: 11 floor: 3 room:4 vid: 1 contagiousDegree: 0
rid: 4 floor: 2 room:1 vid: 1 contagiousDegree: 0
rid: 7 floor: 2 room:4 vid: 1 contagiousDegree: 0
rid: 6 floor: 2 room:3 vid: 1 contagiousDegree: 0
rid: 1 floor: 1 room:2 vid: 1 contagiousDegree: 0
rid: 1 floor: 1 room:2 vid: 2 contagiousDegree: 0.32704249784768646
rid: 6 floor: 2 room:3 vid: 2 contagiousDegree: 0.32704249784768646
rid: 11 floor: 3 room:4 vid: 2 contagiousDegree: 0.32704249784768646
rid: 3 floor: 1 room:4 vid: 2 contagiousDegree: 0.32704249784768646
rid: 2 floor: 1 room:3 vid: 2 contagiousDegree: 0.32704249784768646
rid: 5 floor: 2 room:2 vid: 2 contagiousDegree: 0.32704249784768646
rid: 9 floor: 3 room:2 vid: 2 contagiousDegree: 0.32704249784768646
rid: 0 floor: 1 room:1 vid: 2 contagiousDegree: 0.32704249784768646
rid: 10 floor: 3 room:3 vid: 2 contagiousDegree: 0.32704249784768646
rid: 4 floor: 2 room:1 vid: 2 contagiousDegree: 0.32704249784768646
rid:

In [46]:
print('infected')
pd.DataFrame(infected)

infected


,floor,rid,room
0,1,2,3
1,2,5,2
2,3,9,2
3,1,0,1
4,3,10,3
5,2,7,4


In [47]:
suspiciousVisitors = set([v['visitorId']
                      for visits in itineraries 
                      for v in visits 
                      for i in infected 
                      if v['floor']==i['floor']
                         and v['room']==i['room']])
suspiciousVisitors 

{1, 2, 3, 4, 5}

In [48]:
from collections import Counter
 
countInfectedVisits = Counter([v['visitorId']
                      for visits in itineraries 
                      for v in visits 
                      for i in infected 
                      if v['floor']==i['floor']
                         and v['room']==i['room']])
countInfectedVisits 

Counter({1: 2, 2: 6, 3: 2, 4: 2, 5: 1})

In [49]:
countVisits = Counter([v['visitorId']
                      for visits in itineraries 
                      for v in visits])
countVisits.items()

dict_items([(1, 6), (2, 12), (3, 3), (4, 6), (5, 1)])

In [50]:
ratios = [(vid,vi/v)
         for vid, v in countVisits.items()
         for visitorId, vi in countInfectedVisits.items()
         if vid==visitorId]
ratios

[(1, 0.3333333333333333),
 (2, 0.5),
 (3, 0.6666666666666666),
 (4, 0.3333333333333333),
 (5, 1.0)]

In [51]:
df = pd.DataFrame(ratios)
mainSuspect = df[df.iloc[:,1]==max(df.iloc[:,1])]
mainSuspect

,0,1
4,5,1.0


In [52]:
countVisitsToRoom = Counter([v['rid']
                      for visits in itineraries 
                      for v in visits 
                      for i in infected 
                      if v['floor']==i['floor']
                         and v['room']==i['room']])
countVisitsToRoom

Counter({0: 1, 2: 2, 5: 1, 7: 3, 9: 2, 10: 4})

In [53]:
def visited(visitor_id, room):
    totalVisits = [ tv 
                   for itinerary in itineraries
                   for v in itinerary
                   for rid, tv in countVisitsToRoom.items()
                   if rid == v['rid']
                    and rid == room['rid']
                    and v['visitorId'] == visitor_id]
    return 1/totalVisits[0] if len(totalVisits) > 0 else 0

transitionMatrix = pd.DataFrame(
                    [[visited(v, r) 
                        for r in infected]
                     for v in suspiciousVisitors ],
                    index=suspiciousVisitors,
                    columns=range(1,len(infected)+1))
transitionMatrix

,1,2,3,4,5,6
1,0.0,0.0,0.0,0.0,0.25,0.333333
2,0.5,1.0,0.5,1.0,0.25,0.333333
3,0.0,0.0,0.5,0.0,0.25,0.000000
4,0.5,0.0,0.0,0.0,0.00,0.333333
5,0.0,0.0,0.0,0.0,0.25,0.000000


In [54]:
largestSide = len(transitionMatrix.index) - len(transitionMatrix.columns)
print(largestSide)
if largestSide > 0:
        for i in range(largestSide):
            columnName = 'gr'+ str(i)
            transitionMatrix[columnName] = 1/len(transitionMatrix.index)
elif largestSide < 0:
        for i in range(largestSide*-1):
            print(i)
            row = pd.Series([0 for i in range(len(transitionMatrix.columns))],transitionMatrix.columns)
            transitionMatrix = transitionMatrix.append(row,ignore_index=True)

transitionMatrix

-1
0


,1,2,3,4,5,6
0,0.0,0.0,0.0,0.0,0.25,0.333333
1,0.5,1.0,0.5,1.0,0.25,0.333333
2,0.0,0.0,0.5,0.0,0.25,0.000000
3,0.5,0.0,0.0,0.0,0.00,0.333333
4,0.0,0.0,0.0,0.0,0.25,0.000000
5,0.0,0.0,0.0,0.0,0.00,0.000000


In [55]:
size = len(transitionMatrix.index)
teleportMatrixProbability = np.full((size,size),1/size)
teleportMatrixProbability

array([[ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667],
       [ 0.16666667,  0.16666667,  0.16666667,  0.16666667,  0.16666667,
         0.16666667]])

In [56]:

teleportTransitionMatrix = (0.9*transitionMatrix.as_matrix()) + (0.1*teleportMatrixProbability)
teleportTransitionMatrix

array([[ 0.01666667,  0.01666667,  0.01666667,  0.01666667,  0.24166667,
         0.31666667],
       [ 0.46666667,  0.91666667,  0.46666667,  0.91666667,  0.24166667,
         0.31666667],
       [ 0.01666667,  0.01666667,  0.46666667,  0.01666667,  0.24166667,
         0.01666667],
       [ 0.46666667,  0.01666667,  0.01666667,  0.01666667,  0.01666667,
         0.31666667],
       [ 0.01666667,  0.01666667,  0.01666667,  0.01666667,  0.24166667,
         0.01666667],
       [ 0.01666667,  0.01666667,  0.01666667,  0.01666667,  0.01666667,
         0.01666667]])

In [57]:
currentState = pd.DataFrame(1/size,
                            index=[i for i in range(size)], 
                            columns=[1]).as_matrix()
teleportTransitionMatrix.dot(currentState)

array([[ 0.10416667],
       [ 0.55416667],
       [ 0.12916667],
       [ 0.14166667],
       [ 0.05416667],
       [ 0.01666667]])

In [58]:

print(currentState)
for i in range(0,10000):
    nextState = teleportTransitionMatrix.dot(currentState)
    currentState = nextState
nextState

[[ 0.16666667]
 [ 0.16666667]
 [ 0.16666667]
 [ 0.16666667]
 [ 0.16666667]
 [ 0.16666667]]


array([[ 0.02650538],
       [ 0.86262781],
       [ 0.03910068],
       [ 0.03359409],
       [ 0.02150538],
       [ 0.01666667]])